Tensorflow 및 기타 라이브러리 가져오기

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals
import os
import numpy as np
import matplotlib.pyplot as plt

try:
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

keras = tf.keras

print("tensorflow version",tf.__version__)

In [ ]:
IMG_SIZE = 224 # 모든 이미지는 224x224으로 크기 조정 
EPOCHS = 3
BATCH_SIZE=128
learning_rate = 0.0001

데이터 세트 다운로드 및 탐색

In [ ]:
from keras.datasets import cifar10
from keras.utils import np_utils
import tensorflow_datasets as tfds

tfds.disable_progress_bar()

#분류할 클래스 개수 
num_classes=10 # Cifar10의 클래스 개수

(raw_train, raw_validation, raw_test), metadata = tfds.load(
    'cifar10',
    split=['train[:90%]', 'train[90%:]', 'test'], #test set을 모른다고 가정, train set이 많으면 10%를 val set을 가상으로 만들어 평가를 할것
    with_info=True,
    as_supervised=True,
)

print("Train data 개수:",len(raw_train)) 
print("Val data 개수:",len(raw_validation)) #train에서 떼준 10% -5000
print("Test data 개수:",len(raw_test))

데이터 세트 정규화

In [ ]:
def format_example(image, label): #정규화
  image = tf.cast(image, tf.float32)
  image = (image/127.5) - 1
  image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
  return image, label

In [ ]:
# #map 함수를 사용하여 데이터셋의 각 항목에 데이터 포맷 함수를 적용
train = raw_train.map(format_example) #정규화된 이미지들을train과 val에 넣어줌 
validation = raw_validation.map(format_example)
test = raw_test.map(format_example)

데이터 세트 만들기


In [ ]:
SHUFFLE_BUFFER_SIZE = 1000
train_batches = train.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE) #배치 사이즈만큼 나눠진 train이 들어감
validation_batches = validation.batch(BATCH_SIZE)
test_batches = test.batch(BATCH_SIZE)


데이터 검사하기


In [ ]:
#데이터 가시화
get_label_name = metadata.features['label'].int2str

for image, label in raw_train.take(2):
  plt.figure()
  plt.imshow(image)
  plt.title(get_label_name(label))

사용할 cnn모델 불러오기

In [ ]:
IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3) #가로 세로 채널

#CNN 모델 변경하려면 여기서 변경
#ImageNet으로 사전 훈련된 모델 불러오기 (가중치)
#vgg16 모델 불러옴
base_model = tf.keras.applications.VGG16(input_shape=IMG_SHAPE,
                                               include_top=True, #분류층까지 포함 (특징추출 + 분류층)
                                                classes=1000,
                                               weights='imagenet')

불러온 모델에서 데이터 셋의 클래스 수에 맞게 최종 분류 층 교체


In [ ]:
model = tf.keras.Sequential() 
for layer in base_model.layers[:-1]: # go through until last layer (맨 마지막 레이어만 빼고(최종분류층을 교체해줘야해서) 다 가져와서 반복)
    model.add(layer) 

#마지막 layer의 최종 분류 개수를 클래스 개수와 맞게 설정
#KERAS.DENSE로 fC
model.add(keras.layers.Dense(num_classes, activation='softmax',name='predictions')) #최종분류층을 생성해서 추가해줌

모델 아키텍처 살펴보기


In [ ]:
model.summary()

BLOCK부분은 특징추출
fc1 c2는 분류층
predictions예측

모델 컴파일

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(lr=learning_rate),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy']) #옵티마이저 (adam 사용), loss, metrics(정확도를 볼거임 accuracy) 들을 compile로 지정

모델 훈련


In [ ]:
history = model.fit(train_batches,
                    epochs=EPOCHS,
                    validation_data=validation_batches,
                    batch_size=BATCH_SIZE) #loss와 accuracy가 history에 담김

학습 곡선 그리기

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

test set으로 학습된 모델 테스트

In [ ]:
loss_and_metrics = model.evaluate(test_batches, batch_size=64)
print("테스트 성능 : {}%".format(round(loss_and_metrics[1]*100,4))) #accuracy를 구할거니까 구한 값의 인덱스 1을 지정해 불러옴

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!jupyter nbconvert --to html "/content/drive/MyDrive/Colab Notebooks/VGGNet_train_tensorflow .ipynb"